# MScBMI 33200 – Machine Learning for Biomedical Informatics
# Assignment I

Name: Troy Zhongyi Zhang      
Netid: zhongyiz@uchicago.edu      

#### Directions        
1. Follow instructions below for each question      
2. You can use either R or Python for completing the assignment      
3. Upload your answer sheet along with your code (as HTML or PDF) in a separate file. R users can Knit an R markdown into HTML/PDF. Python users can use IPython or Jupyer notebooks and convert them into HTML/PDF (see instructions )      

# Section 1: EMR Bots 30-day Readmission study

### Read Data (Preparation)

In [1]:
import pandas as pd
import numpy as np
from tableone import TableOne
info = pd.read_csv("encounter_info.csv")
labs = pd.read_csv("encounter_labs.csv")
df = pd.read_csv("readmission_outcome.csv")

info.shape

(36143, 7)

In [2]:
labs.shape

(4902804, 18)

In [3]:
df.shape

(36143, 2)

# Section 1, (Q1):

In [4]:
df1 = pd.merge(info, df, on="Encounter_ID")

In [5]:
df1.head()

,Patient_ID,Encounter_ID,AdmissionStartDate,AdmissionEndDate,PatientGender,PatientRace,PatientEncounterAge,outcome
0,100000,100000_2,1996-01-20 14:32:48.440,1996-02-06 17:11:05.247,Female,White,36.456455,0
1,100000,100000_1,1981-01-06 01:54:13.577,1981-01-08 23:35:05.233,Female,White,21.408437,0
2,100000,100000_5,2007-04-12 03:30:56.917,2007-04-27 18:30:58.473,Female,White,47.688073,0
3,100000,100000_3,2002-04-06 23:17:11.963,2002-04-11 00:49:49.810,Female,White,42.671151,0
4,100000,100000_7,2012-07-18 12:01:05.853,2012-07-21 04:17:51.173,Female,White,52.960276,0


In [6]:
columns = ['PatientEncounterAge','PatientGender','PatientRace']
categorical = ['PatientGender','PatientRace']
groupby = 'outcome'
mytable = TableOne(df1, columns, categorical, groupby, pval=True)
print(mytable)

                                     Grouped by outcome                                                     
                                                 isnull             0            1   pval              ptest
variable            level                                                                                   
n                                                               36015          128                          
PatientEncounterAge                                   0   41.7 (18.1)  44.3 (18.0)  0.108  Two Sample T-test
PatientGender       Female                            0  18812 (52.2)    64 (50.0)  0.677        Chi-squared
                    Male                                 17203 (47.8)    64 (50.0)                          
PatientRace         African American                  0   5382 (14.9)    21 (16.4)  0.580        Chi-squared
                    Asian                                 8251 (22.9)    33 (25.8)                          
                   

In [7]:
from scipy.stats import ttest_ind
df30 = df1[df1['outcome']==1]
dfnot30 = df1[df1['outcome']==0]
ttest_ind(df30['PatientEncounterAge'], dfnot30['PatientEncounterAge'])

Ttest_indResult(statistic=1.6180629064428116, pvalue=0.10565776793068646)

# Section 1, (Q2):

### Find out the latest lab dates for each encounter

In [8]:
lab=labs.copy()

In [9]:
lab['Lab_DTTM'] = pd.to_datetime(lab['Lab_DTTM'])

In [10]:
grouped_lab = lab.sort_values("Lab_DTTM", ascending=False
                             ).groupby("Encounter_ID")["Lab_DTTM"
                                                      ].apply(list).apply(pd.Series
                                                                         ).reset_index()
grouped_lab = grouped_lab[["Encounter_ID", 0]]
grouped_lab.columns = ["Encounter_ID", "Lab_DTTM"]

In [11]:
grouped_lab.shape

(36143, 2)

In [12]:
newlab0 = pd.merge(grouped_lab, lab, on=['Encounter_ID','Lab_DTTM'])

In [13]:
newlab0.shape

(36173, 18)

### Only keep the last lab data row for each encounter

In [14]:
newlab0 = newlab0.drop_duplicates('Encounter_ID', keep='last').values
newlab0.shape

(36143, 18)

In [15]:
newlab0 = pd.DataFrame(newlab0)

In [16]:
col_name = list(lab.columns.values)
newlab0.columns = col_name

In [17]:
newlab02 = newlab0.iloc[36136:]

In [18]:
newlab02['Encounter_ID'] = pd.DataFrame(newlab02['Encounter_ID']).applymap(lambda x: x.replace('1e+05', '100000'))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [19]:
newlab00 = newlab0.replace(newlab02)

In [20]:
newlab00.describe()

,CBC..ABSOLUTE.LYMPHOCYTES,CBC..ABSOLUTE.NEUTROPHILS,CBC..BASOPHILS,CBC..EOSINOPHILS,CBC..HEMATOCRIT,CBC..HEMOGLOBIN,CBC..PLATELET.COUNT,CBC..RED.BLOOD.CELL.COUNT,CBC..WHITE.BLOOD.CELL.COUNT,METABOLIC..ALBUMIN,METABOLIC..BILI.TOTAL,METABOLIC..BUN,METABOLIC..CALCIUM,METABOLIC..CREATININE,METABOLIC..POTASSIUM,METABOLIC..SODIUM
count,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000
mean,25.012467,70.014382,0.109371,0.340046,42.538489,14.508951,284.655720,4.998442,7.518798,4.249077,0.602128,17.510342,9.501541,0.851078,4.491819,140.038799
std,5.773137,5.765241,0.073498,0.154809,7.173616,2.598711,95.354652,1.156924,2.597528,1.013476,0.349107,7.183214,1.443756,0.206359,0.867464,8.668622
min,15.000000,60.000000,0.000000,0.100000,30.000000,10.000000,120.000000,3.000000,3.000000,2.500000,0.000000,5.000000,7.000000,0.500000,3.000000,125.000000
25%,20.000000,65.000000,0.100000,0.200000,36.400000,12.300000,202.100000,4.000000,5.200000,3.400000,0.300000,11.400000,8.300000,0.700000,3.700000,132.500000
50%,25.000000,70.000000,0.100000,0.300000,42.600000,14.500000,284.400000,5.000000,7.500000,4.200000,0.600000,17.500000,9.500000,0.900000,4.500000,140.100000
75%,30.000000,75.000000,0.200000,0.500000,48.700000,16.800000,367.100000,6.000000,9.800000,5.100000,0.900000,23.700000,10.700000,1.000000,5.200000,147.600000
max,35.000000,80.000000,0.200000,0.600000,55.000000,19.000000,450.000000,7.000000,12.000000,6.000000,1.200000,30.000000,12.000000,1.200000,6.000000,155.000000


### Merge the three datasets into one correspondingly

In [21]:
q2 = pd.merge(info, newlab00, on=['Encounter_ID'],how='left').merge(df, on=['Encounter_ID'],how='left')

#### Drop redundant columns

In [22]:
q2.drop(['Patient_ID', 'AdmissionStartDate', 'AdmissionEndDate', 'Lab_DTTM'], axis=1, inplace=True)

In [23]:
q2

,Encounter_ID,PatientGender,PatientRace,PatientEncounterAge,CBC..ABSOLUTE.LYMPHOCYTES,CBC..ABSOLUTE.NEUTROPHILS,CBC..BASOPHILS,CBC..EOSINOPHILS,CBC..HEMATOCRIT,CBC..HEMOGLOBIN,...,CBC..RED.BLOOD.CELL.COUNT,CBC..WHITE.BLOOD.CELL.COUNT,METABOLIC..ALBUMIN,METABOLIC..BILI.TOTAL,METABOLIC..BUN,METABOLIC..CALCIUM,METABOLIC..CREATININE,METABOLIC..POTASSIUM,METABOLIC..SODIUM,outcome
0,100000_2,Female,White,36.456455,16.1,77.9,0.2,0.4,49.6,16.2,...,5.8,6.4,2.7,0.9,15.6,8.1,1.2,4.7,140.9,0
1,100000_1,Female,White,21.408437,32.8,76.4,0.0,0.1,35.9,14.8,...,4.8,10.7,5.8,0.8,12.6,11.1,0.9,4.1,135.5,0
2,100000_5,Female,White,47.688073,27.4,62.2,0.1,0.4,45.2,16.8,...,4.9,11.8,5.9,0.1,21.4,7.5,0.8,5.6,137.1,0
3,100000_3,Female,White,42.671151,23.4,66.2,0.1,0.4,40.4,18.7,...,3.2,8.5,3.7,0.3,15.1,7.9,0.7,5.3,150.0,0
4,100000_7,Female,White,52.960276,30.3,64.4,0.1,0.5,38.9,18.6,...,5.3,9.1,4.1,1.2,5.2,7.2,1.2,5.7,151.7,0
5,100000_4,Female,White,47.277525,15.7,78.3,0.0,0.2,54.4,18.9,...,5.9,7.0,4.0,0.9,24.4,7.0,1.1,4.4,155.0,0
6,100000_6,Female,White,48.049906,27.1,66.1,0.1,0.6,39.1,12.8,...,6.8,8.0,4.8,1.1,27.7,8.4,0.7,3.4,139.5,0
7,100001_1,Female,African American,23.467454,33.3,67.2,0.2,0.4,40.2,18.1,...,6.6,4.7,3.5,1.2,20.5,11.3,1.1,4.4,137.6,0
8,100002_2,Female,Unknown,43.045790,17.2,70.7,0.0,0.3,37.9,11.7,...,3.3,9.2,3.4,0.7,14.7,11.9,1.2,5.7,129.1,0
9,100002_1,Female,Unknown,27.803538,29.4,64.5,0.0,0.3,42.9,13.4,...,5.9,6.4,3.2,0.0,25.9,11.8,0.7,3.3,145.2,0


In [24]:
q2.describe()

,PatientEncounterAge,CBC..ABSOLUTE.LYMPHOCYTES,CBC..ABSOLUTE.NEUTROPHILS,CBC..BASOPHILS,CBC..EOSINOPHILS,CBC..HEMATOCRIT,CBC..HEMOGLOBIN,CBC..PLATELET.COUNT,CBC..RED.BLOOD.CELL.COUNT,CBC..WHITE.BLOOD.CELL.COUNT,METABOLIC..ALBUMIN,METABOLIC..BILI.TOTAL,METABOLIC..BUN,METABOLIC..CALCIUM,METABOLIC..CREATININE,METABOLIC..POTASSIUM,METABOLIC..SODIUM,outcome
count,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000,36143.000000
mean,41.748503,25.012467,70.014382,0.109371,0.340046,42.538489,14.508951,284.655720,4.998442,7.518798,4.249077,0.602128,17.510342,9.501541,0.851078,4.491819,140.038799,0.003541
std,18.058317,5.773137,5.765241,0.073498,0.154809,7.173616,2.598711,95.354652,1.156924,2.597528,1.013476,0.349107,7.183214,1.443756,0.206359,0.867464,8.668622,0.059406
min,18.011880,15.000000,60.000000,0.000000,0.100000,30.000000,10.000000,120.000000,3.000000,3.000000,2.500000,0.000000,5.000000,7.000000,0.500000,3.000000,125.000000,0.000000
25%,25.839463,20.000000,65.000000,0.100000,0.200000,36.400000,12.300000,202.100000,4.000000,5.200000,3.400000,0.300000,11.400000,8.300000,0.700000,3.700000,132.500000,0.000000
50%,38.588578,25.000000,70.000000,0.100000,0.300000,42.600000,14.500000,284.400000,5.000000,7.500000,4.200000,0.600000,17.500000,9.500000,0.900000,4.500000,140.100000,0.000000
75%,54.398018,30.000000,75.000000,0.200000,0.500000,48.700000,16.800000,367.100000,6.000000,9.800000,5.100000,0.900000,23.700000,10.700000,1.000000,5.200000,147.600000,0.000000
max,92.958042,35.000000,80.000000,0.200000,0.600000,55.000000,19.000000,450.000000,7.000000,12.000000,6.000000,1.200000,30.000000,12.000000,1.200000,6.000000,155.000000,1.000000


In [25]:
q2.dtypes

Encounter_ID                    object
PatientGender                   object
PatientRace                     object
PatientEncounterAge            float64
CBC..ABSOLUTE.LYMPHOCYTES      float64
CBC..ABSOLUTE.NEUTROPHILS      float64
CBC..BASOPHILS                 float64
CBC..EOSINOPHILS               float64
CBC..HEMATOCRIT                float64
CBC..HEMOGLOBIN                float64
CBC..PLATELET.COUNT            float64
CBC..RED.BLOOD.CELL.COUNT      float64
CBC..WHITE.BLOOD.CELL.COUNT    float64
METABOLIC..ALBUMIN             float64
METABOLIC..BILI.TOTAL          float64
METABOLIC..BUN                 float64
METABOLIC..CALCIUM             float64
METABOLIC..CREATININE          float64
METABOLIC..POTASSIUM           float64
METABOLIC..SODIUM              float64
outcome                          int64
dtype: object

# Section 1, (Q3):

In [26]:
newlab03 = newlab00.copy()

In [27]:
newlab03['mean_labs']= newlab03.iloc[:, -16:].sum(axis=1)/16

In [28]:
newlab03 = newlab03[['Encounter_ID','mean_labs']]

In [29]:
q3 = pd.merge(info, newlab03, on=['Encounter_ID'],how='left').merge(df, on=['Encounter_ID'],how='left')
q3.drop(['Patient_ID', 'AdmissionStartDate', 'AdmissionEndDate'], axis=1, inplace=True)
q3.describe()

,PatientEncounterAge,mean_labs,outcome
count,36143.000000,36143.000000,36143.000000
mean,41.748503,39.183841,0.003541
std,18.058317,6.047758,0.059406
min,18.011880,26.262500,0.000000
25%,25.839463,34.009375,0.000000
50%,38.588578,39.143750,0.000000
75%,54.398018,44.356250,0.000000
max,92.958042,52.781250,1.000000


In [30]:
q3.head()

,Encounter_ID,PatientGender,PatientRace,PatientEncounterAge,mean_labs,outcome
0,100000_2,Female,White,36.456455,42.86875,0
1,100000_1,Female,White,21.408437,45.76875,0
2,100000_5,Female,White,47.688073,41.41250,0
3,100000_3,Female,White,42.671151,47.03750,0
4,100000_7,Female,White,52.960276,43.47500,0


In [31]:
q3.dtypes

Encounter_ID            object
PatientGender           object
PatientRace             object
PatientEncounterAge    float64
mean_labs              float64
outcome                  int64
dtype: object

# Section 1, (Q4):

In [32]:
df30.head()

,Patient_ID,Encounter_ID,AdmissionStartDate,AdmissionEndDate,PatientGender,PatientRace,PatientEncounterAge,outcome
373,100103,100103_1,1982-06-29 21:16:48.833,1982-07-12 01:09:45.967,Male,Asian,18.133936,1
433,100118,100118_5,2005-06-15 14:36:49.753,2005-06-29 12:55:34.900,Male,White,49.796189,1
474,100131,100131_1,1980-10-06 00:29:48.673,1980-10-11 08:05:28.010,Female,White,26.301427,1
637,100174,100174_1,1991-09-11 18:28:18.557,1991-09-24 20:24:47.230,Male,African American,25.933616,1
1169,100331,100331_6,2007-12-07 23:28:17.110,2007-12-16 16:44:45.767,Male,Unknown,75.158844,1


In [33]:
df30s04 = df30[df30["AdmissionEndDate"].str[:4].astype(int)<=2004]

In [34]:
df30g04 = df30[df30["AdmissionEndDate"].str[:4].astype(int)>2004]

In [35]:
# Patients who were re-admitted within 30 days and years <= 2004
len(df30s04.index)

78

In [36]:
# Patients who were re-admitted within 30 days and years > 2004
len(df30g04.index)

50

In [37]:
# Patients who were not re-admitted within 30 days and years <= 2004
dfnot30s04 = dfnot30[dfnot30["AdmissionEndDate"].str[:4].astype(int)<=2004]
dfnot30g04 = dfnot30[dfnot30["AdmissionEndDate"].str[:4].astype(int)>2004]
len(dfnot30s04.index)

21416

In [38]:
# Patients who were not re-admitted within 30 days and years > 2004
len(dfnot30g04.index)

14599

#### Total for the training with percentage

In [39]:
len(df30s04.index)+len(dfnot30s04.index)

21494

In [40]:
print(len(df30s04.index)/(len(df30s04.index)+len(dfnot30s04.index))*100,"%")

0.36289196985205174 %


In [41]:
print(len(dfnot30s04.index)/(len(df30s04.index)+len(dfnot30s04.index))*100,"%")

99.63710803014794 %


#### Total for the test set with percentage

In [42]:
len(df30g04.index)+len(dfnot30g04.index)

14649

In [43]:
print(len(df30g04.index)/(len(df30g04.index)+len(dfnot30g04.index))*100,"%")

0.34132022663663053 %


In [44]:
print(len(dfnot30g04.index)/(len(df30g04.index)+len(dfnot30g04.index))*100,"%")

99.65867977336337 %


-----------------------

# Section 2: GUSTO 30-day Mortality Prediction

## Data preparation

In [46]:
gusto = pd.read_csv("gusto_data.csv")
gusto.head()

,DAY30,AGE,A65,SEX,KILLIP,SHO,DIA,HYP,HRT,ANT,...,WEI,SMK,HTN,LIP,PAN,FAM,STE,ST4,TTR,GROUP
0,0,70.313,1,0,1,0,0,0,0,1,...,84.0,3,1,1,0,0,1,0,1,west
1,0,59.844,0,0,1,0,1,0,0,1,...,115.0,1,1,0,0,1,6,1,0,west
2,0,59.023,0,0,1,0,0,0,1,0,...,76.0,1,1,0,0,1,3,0,0,west
3,1,80.375,1,1,1,0,0,0,1,0,...,50.0,3,0,0,0,0,3,0,0,west
4,0,64.750,0,0,1,0,0,1,0,0,...,97.4,1,0,0,1,1,2,0,1,west


In [47]:
gusto.shape

(3661, 23)

In [48]:
gusto0=gusto.copy()
gusto0.dropna()
gusto0.shape

(3661, 23)

# Section 2, (Q1):

### Total number

In [49]:
columns2 = ['AGE','SEX','GROUP','DAY30']
categorical2 = ['SEX','GROUP']
groupby2 = 'DAY30'
mytable2 = TableOne(gusto, columns2, categorical2, groupby2, pval=True)
print(mytable2)

                 Grouped by DAY30                                                     
                           isnull            0            1    pval              ptest
variable level                                                                        
n                                         3430          231                           
AGE                             0  60.2 (11.6)  71.1 (10.7)  <0.001  Two Sample T-test
SEX      0                      0  2581 (75.2)   140 (60.6)  <0.001        Chi-squared
         1                          849 (24.8)    91 (39.4)                           
GROUP    sample2                0    239 (7.0)     20 (8.7)   0.694        Chi-squared
         sample4                    733 (21.4)    52 (22.5)                           
         sample5                    405 (11.8)    24 (10.4)                           
         west                      2053 (59.9)   135 (58.4)                           
[1] Warning, test for normality reports non

In [50]:
g30 = gusto[gusto['DAY30']==1]
gnot30 = gusto[gusto['DAY30']==0]
ttest_ind(g30['AGE'], gnot30['AGE'])

Ttest_indResult(statistic=13.957560682198121, pvalue=3.5536013952145326e-43)

# Section 2, (Q2):

#### Patients who died within 30 days  

In [51]:
len(g30.loc[(g30['GROUP'] == 'sample2'
            ) | (g30['GROUP'] == 'sample4'
                ) | (g30['GROUP'] == 'sample5')].index)

96

In [52]:
len(g30[g30['GROUP']=='west'].index)

135

#### Patients who were alive at 30 days

In [53]:
len(gnot30.loc[(gnot30['GROUP'] == 'sample2'
               ) | (gnot30['GROUP'] == 'sample4'
                   ) | (gnot30['GROUP'] == 'sample5')].index)

1377

In [54]:
len(gnot30[gnot30['GROUP']=='west'].index)

2053

#### Total and percentage for training

In [55]:
len(g30.loc[(g30['GROUP'] == 'sample2'
            ) | (g30['GROUP'] == 'sample4'
                ) | (g30['GROUP'] == 'sample5')].index)+len(gnot30.loc[(gnot30['GROUP'] == 'sample2'
               ) | (gnot30['GROUP'] == 'sample4'
                   ) | (gnot30['GROUP'] == 'sample5')].index)

1473

In [56]:
print(len(g30.loc[(g30['GROUP'] == 'sample2'
                  ) | (g30['GROUP'] == 'sample4'
                      ) | (g30['GROUP'] == 'sample5'
                          )].index)/(len(g30.loc[(g30['GROUP'] == 'sample2'
                                                 ) | (g30['GROUP'] == 'sample4'
                                                     ) | (g30['GROUP'] == 'sample5'
                                                         )].index)+len(
    gnot30.loc[(gnot30['GROUP'] == 'sample2'
               ) | (gnot30['GROUP'] == 'sample4'
                   ) | (gnot30['GROUP'] == 'sample5'
                       )].index))*100,"%")


6.517311608961303 %


In [57]:
print(len(gnot30.loc[(gnot30['GROUP'] == 'sample2'
                     ) | (gnot30['GROUP'] == 'sample4'
                         ) | (gnot30['GROUP'] == 'sample5'
                             )].index)/(len(g30.loc[(g30['GROUP'] == 'sample2'
                                                    ) | (g30['GROUP'] == 'sample4'
                                                        ) | (g30['GROUP'] == 'sample5'
                                                            )].index)+len(
    gnot30.loc[(gnot30['GROUP'] == 'sample2'
               ) | (gnot30['GROUP'] == 'sample4'
                   ) | (gnot30['GROUP'] == 'sample5'
                       )].index))*100,"%")


93.48268839103869 %


#### Total and percentage for test set

In [58]:
len(g30[g30['GROUP']=='west'].index)+len(gnot30[gnot30['GROUP']=='west'].index)

2188

In [59]:
print(len(g30[g30['GROUP']=='west'
             ].index)/(len(g30[g30['GROUP']=='west'
                              ].index)+len(gnot30[gnot30['GROUP'
                                                        ]=='west'].index))*100,"%")

6.170018281535649 %


In [60]:
print(len(gnot30[gnot30['GROUP']=='west'
                ].index)/(len(g30[g30['GROUP']=='west'
                                 ].index)+len(gnot30[gnot30['GROUP'
                                                           ]=='west'].index))*100,"%")

93.82998171846435 %


# Section 2, (Q3):

In [61]:
g30_train = g30.loc[(g30['GROUP'] == 'sample2'
                    ) | (g30['GROUP'] == 'sample4'
                        ) | (g30['GROUP'] == 'sample5')]
gnot30_train = gnot30.loc[(gnot30['GROUP'] == 'sample2'
                          ) | (gnot30['GROUP'] == 'sample4'
                              ) | (gnot30['GROUP'] == 'sample5')]
gusto_train = pd.concat([g30_train,gnot30_train])
gusto_train = gusto_train.reset_index(drop=True)
gusto_train.describe()

,DAY30,AGE,A65,SEX,KILLIP,SHO,DIA,HYP,HRT,ANT,...,HEI,WEI,SMK,HTN,LIP,PAN,FAM,STE,ST4,TTR
count,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,...,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000,1473.000000
mean,0.065173,61.415623,0.410726,0.268839,1.194840,0.020367,0.114732,0.073320,0.292600,0.361168,...,170.338900,78.200068,1.909029,0.385608,0.386286,0.364562,0.428377,4.150034,0.386965,0.560760
std,0.246915,11.448781,0.492133,0.443507,0.462655,0.141299,0.318806,0.260749,0.455111,0.480502,...,9.779777,16.531963,0.803275,0.486904,0.487063,0.481471,0.495012,1.865345,0.487221,0.496463
min,0.000000,25.891000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,141.000000,37.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,52.578000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,163.800000,68.000000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000
50%,0.000000,62.242000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,170.200000,77.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000
75%,0.000000,70.469000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,177.300000,87.000000,3.000000,1.000000,1.000000,1.000000,1.000000,6.000000,1.000000,1.000000
max,1.000000,88.828000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,199.700000,180.000000,3.000000,1.000000,1.000000,1.000000,1.000000,10.000000,1.000000,1.000000


In [62]:
gusto_train.dtypes

DAY30       int64
AGE       float64
A65         int64
SEX         int64
KILLIP      int64
SHO         int64
DIA         int64
HYP         int64
HRT         int64
ANT         int64
PMI         int64
HIG         int64
HEI       float64
WEI       float64
SMK         int64
HTN         int64
LIP         int64
PAN         int64
FAM         int64
STE         int64
ST4         int64
TTR         int64
GROUP      object
dtype: object

In [63]:
g30_test = g30[g30['GROUP']=='west']
gnot30_test = gnot30[gnot30['GROUP']=='west']
gusto_test = pd.concat([g30_test,gnot30_test])
gusto_test = gusto_test.reset_index(drop=True)
gusto_test.describe()

,DAY30,AGE,A65,SEX,KILLIP,SHO,DIA,HYP,HRT,ANT,...,HEI,WEI,SMK,HTN,LIP,PAN,FAM,STE,ST4,TTR
count,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,...,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000,2188.000000
mean,0.061700,60.469186,0.383455,0.248629,1.132084,0.014625,0.142596,0.096435,0.333638,0.372486,...,172.129936,82.888940,1.866545,0.403565,0.404936,0.340494,0.475777,3.999543,0.356033,0.608775
std,0.240665,12.026568,0.486339,0.432317,0.409550,0.120075,0.349740,0.295254,0.471620,0.483577,...,10.094343,17.692498,0.821252,0.490724,0.490992,0.473984,0.499527,1.878451,0.478935,0.488136
min,0.000000,23.910000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,140.900000,36.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,50.932000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,165.100000,70.900000,1.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000
50%,0.000000,60.547000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,173.000000,82.000000,2.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,1.000000
75%,0.000000,69.922000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,...,180.000000,92.050000,3.000000,1.000000,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000
max,1.000000,89.484000,1.000000,1.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,205.700000,180.000000,3.000000,1.000000,1.000000,1.000000,1.000000,11.000000,1.000000,1.000000


In [64]:
gusto_test.dtypes

DAY30       int64
AGE       float64
A65         int64
SEX         int64
KILLIP      int64
SHO         int64
DIA         int64
HYP         int64
HRT         int64
ANT         int64
PMI         int64
HIG         int64
HEI       float64
WEI       float64
SMK         int64
HTN         int64
LIP         int64
PAN         int64
FAM         int64
STE         int64
ST4         int64
TTR         int64
GROUP      object
dtype: object

--------------------------